In [139]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [140]:
# IMPORT LIB
import os.path
from pathlib import Path

# PIP
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *
from tensorflow.keras.models import Model

In [141]:
image_dir = Path("/content/drive/MyDrive/MathML/A2_CNN/Images")

In [142]:
# Load Dataset
# Folder: The FilePath name, Carrots, Rockets, will be labels for each image.
filepaths = list(image_dir.glob(r'**/*.jpg')) # jpg files in any folders
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)

# Mix the dataset randomly
image_df = image_df.sample(frac=1).reset_index(drop = True)
image_df.head(8)

,Filepath,Label
0,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
1,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
2,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
3,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
4,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
5,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
6,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
7,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets


In [143]:
image_df

,Filepath,Label
0,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
1,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
2,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
3,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
4,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
...,...,...
301,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
302,/content/drive/MyDrive/MathML/A2_CNN/Images/Ca...,Carrots
303,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets
304,/content/drive/MyDrive/MathML/A2_CNN/Images/Ro...,Rockets


In [144]:
# Image Pre-processing => Augmentation is required since too few images
train_generator = ImageDataGenerator(
    rescale=1./255, # image -> 0~1 normalization
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.1 # 10% of dataset is allocated to validation dataset
)

In [145]:
# Image file path to image dataset
train_images = train_generator.flow_from_dataframe(
    dataframe=image_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical', # Classification
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=image_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

Found 276 validated image filenames belonging to 2 classes.
Found 30 validated image filenames belonging to 2 classes.


In [146]:
num_classes = len(image_df['Label'].unique())

In [150]:
# Load ResNet50 (Pre-trained weights)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# To prevent overfitting, base_model won't be learning.
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for your specific task
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer

# Additional custom layers
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)

# Output layer for your specific task
outputs = Dense(num_classes, activation='sigmoid')(x)

# Create the final Model
model = Model(inputs=base_model.input, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
'''
# ResNet Model (Pre-trained weights)
base_model = ResNet50(weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(1024, activation='relu')(x)  # Custom fully-connected layer
predictions = Dense(num_classes, activation='softmax')(x)  # Output layer with softmax activation

# Create the final fine-tuned model
model = Model(inputs=base_model.input, outputs=predictions)

# fine-tuning
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
'''

In [151]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=20,
)

Epoch 1/20
9/9 [==============================] - 10s 761ms/step - loss: 0.8720 - accuracy: 0.4928 - val_loss: 0.7632 - val_accuracy: 0.4000
Epoch 2/20
9/9 [==============================] - 6s 614ms/step - loss: 0.7339 - accuracy: 0.5145 - val_loss: 0.7017 - val_accuracy: 0.4000
Epoch 3/20
9/9 [==============================] - 6s 622ms/step - loss: 0.7334 - accuracy: 0.5109 - val_loss: 0.6740 - val_accuracy: 0.6000
Epoch 4/20
9/9 [==============================] - 6s 619ms/step - loss: 0.7312 - accuracy: 0.5435 - val_loss: 0.6670 - val_accuracy: 0.6333
Epoch 5/20
9/9 [==============================] - 6s 649ms/step - loss: 0.7024 - accuracy: 0.5507 - val_loss: 0.6629 - val_accuracy: 0.6333
Epoch 6/20
9/9 [==============================] - 6s 650ms/step - loss: 0.6828 - accuracy: 0.5652 - val_loss: 0.6380 - val_accuracy: 0.6333
Epoch 7/20
9/9 [==============================] - 6s 621ms/step - loss: 0.6715 - accuracy: 0.5906 - val_loss: 0.6285 - val_accuracy: 0.7333
Epoch 8/20
9/9 [===